1回目 モデルを作ってベースラインをとる

数値データのみ
Null無しデータのみ
K-最近ぼうぼうを使う



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/standard/titanic

/content/drive/My Drive/Colab Notebooks/standard/titanic


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# データの読み込み
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')  # 今回は使用しない

# 特徴量と目的変数の準備
features = train_df[['Pclass', 'SibSp', 'Parch']]
target = train_df['Survived']  # Seriesとして取得

# 学習用データとテスト用データに分割
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=42
)

# K近傍法モデルのインスタンス化と学習
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(features_train, target_train)

# スコア（精度）の表示
score = knn_classifier.score(features_test, target_test)
print(f"KNNモデルのスコア: {score:.4f}")


KNNモデルのスコア: 0.6780


ここから2回目の改修 有用な特徴量を探す

カテゴリ特徴量をワンホットエンコーディング
スケール変換
Nullを埋める




In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

train_df = pd.read_csv('data/train.csv')

# 特徴量と目的変数の準備
features = train_df[['Pclass', 'SibSp', 'Parch', 'Sex']]
target = train_df[['Survived']]

# カテゴリ変数をダミー変数化
features = pd.get_dummies(features, columns=['Pclass', 'Sex'])
print(features.head())

# 学習用データとテスト用データに分割
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=42
)

# K近傍法モデルのインスタンス化と学習
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(features_train, target_train)

# スコア（精度）の表示
score = knn_classifier.score(features_test, target_test)
print(f"KNNモデルのスコア: {score:.4f}")


   SibSp  Parch  Pclass_1  Pclass_2  Pclass_3  Sex_female  Sex_male
0      1      0     False     False      True       False      True
1      1      0      True     False     False        True     False
2      0      0     False     False      True        True     False
3      1      0      True     False     False        True     False
4      0      0     False     False      True       False      True
KNNモデルのスコア: 0.7695


/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


ここから3回目の改修

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')  # 提出用データの作成に使用

# 特徴量と目的変数の準備
features = train_df[['Pclass', 'SibSp', 'Parch', 'Sex', 'Fare']]
target = train_df[['Survived']]

# カテゴリ変数をダミー変数化
features = pd.get_dummies(features, columns=['Pclass', 'Sex'])

# 学習用データとテスト用データに分割
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=42
)

# ------------------------
# (1) スケーリング前のKNN
# ------------------------
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(features_train, target_train)

score_before_scaling = knn_classifier.score(features_test, target_test)
print(f"KNNスコア(スケーリング前): {score_before_scaling:.4f}")

# ------------------------
# (2) スケーリング後のKNN
# ------------------------
scaler = StandardScaler()
scaler.fit(features_train)

# スケーリング
features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

knn_classifier_scaled = KNeighborsClassifier()
knn_classifier_scaled.fit(features_train_scaled, target_train)

score_after_scaling = knn_classifier_scaled.score(features_test_scaled, target_test)
print(f"KNNスコア(スケーリング後): {score_after_scaling:.4f}")

# ------------------------
# (3) 提出用データを予測
# ------------------------
features_submit = test_df[['Pclass', 'SibSp', 'Parch', 'Sex', 'Fare']]
submission_df = test_df[['PassengerId']].copy()

# テスト用データと同じ前処理（ダミー変数化 & スケーリング）
features_submit = pd.get_dummies(features_submit, columns=['Pclass', 'Sex'])
features_submit['Fare'] = features_submit['Fare'].fillna(features_submit['Fare'].mean())

# 特徴量の列が学習時と同じ形になるようにカラムを合わせることに注意
# （本来はtrainとtestを結合してpd.get_dummiesするなどの対応が必要）
# ここでは簡易的に、そのままfit済みのscalerを適用
features_submit_scaled = scaler.transform(features_submit)

# 予測
submission_df['Survived'] = knn_classifier_scaled.predict(features_submit_scaled)

submission_df.head()


KNNスコア(スケーリング前): 0.7797
KNNスコア(スケーリング後): 0.7864


/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.11/dist-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


ここから4回目 他のモデルを試して精度を確認する

ロジスティック回帰
サポートベクターマシーン
決定木
ランダムフォレスト
勾配ぶースティング回帰木


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

train_df = pd.read_csv('data/train.csv')

# 特徴量と目的変数
features = train_df[['Pclass', 'SibSp', 'Parch', 'Sex', 'Fare']]
target = train_df[['Survived']]

# ダミー変数化
features = pd.get_dummies(features, columns=['Pclass', 'Sex'])

# 学習用/テスト用データに分割
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=42
)

# スケーリング
scaler = StandardScaler()
scaler.fit(features_train)

features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

# LinearSVC のインスタンス化＆学習
svc_classifier = LinearSVC()
svc_classifier.fit(features_train_scaled, target_train)

# スコア（精度）を表示
score = svc_classifier.score(features_test_scaled, target_test)
print(f"LinearSVCのスコア: {score:.4f}")


LinearSVCのスコア: 0.7966


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 必要ならインポート
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

train_df = pd.read_csv('data/train.csv')

# 特徴量と目的変数
features = train_df[['Pclass', 'SibSp', 'Parch', 'Sex', 'Fare']]
target = train_df[['Survived']]

# ダミー変数化
features = pd.get_dummies(features, columns=['Pclass', 'Sex'])

# 学習用/テスト用データに分割
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=42
)

# スケーリング
scaler = StandardScaler()
scaler.fit(features_train)

features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

# ------------------------
# GradientBoostingClassifier
# ------------------------
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(features_train_scaled, target_train)

gb_score = gb_classifier.score(features_test_scaled, target_test)
print(f"GradientBoostingClassifierのスコア: {gb_score:.4f}")

# ------------------------
# RandomForestClassifier
# ------------------------
rf_classifier = RandomForestClassifier()
rf_classifier.fit(features_train_scaled, target_train)

rf_score = rf_classifier.score(features_test_scaled, target_test)
print(f"RandomForestClassifierのスコア: {rf_score:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifierのスコア: 0.8169


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifierのスコア: 0.7763


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

# ------------------------
# 1. データ読み込み
# ------------------------
train_df = pd.read_csv('data/train.csv')  # KaggleのTitanic練習用データ
test_df = pd.read_csv('data/test.csv')    # 提出用（今回精度計測には使わない）
# 実際のKaggle提出をする場合は、test_dfにも同様の前処理を行う必要があります。

# ------------------------
# 2. 特徴量エンジニアリング
# ------------------------
# (A) Age の欠損値補完（平均 or 中央値）
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)

# (B) Embarked の欠損補完 & ダミー変数化
# もし欠損値があれば、とりあえず 'S' などにしておく
train_df['Embarked'].fillna('S', inplace=True)

# (C) Title (敬称) の抽出
# 例: "Braund, Mr. Owen Harris" → "Mr"
def extract_title(name):
    # ピリオドの直前にある単語を取る方法が定番
    # 例: "Mr.", "Mrs.", "Miss."
    # ただし "Countess." "Dr." "Rev." などレアなものがあるのでマッピング処理を入れる
    title = name.split(",")[1].split(".")[0].strip()
    return title

train_df['Title'] = train_df['Name'].apply(extract_title)

# レアタイトルを一括りにするなどの前処理
common_titles = ['Mr', 'Mrs', 'Miss', 'Master']
train_df.loc[~train_df['Title'].isin(common_titles), 'Title'] = 'Others'

# 使いたい列を抽出
# (すでに "Age", "Embarked", "Title" を追加)
# fare は欠損がないはずだが、念のため fillna で埋めておく例
train_df['Fare'].fillna(train_df['Fare'].median(), inplace=True)

features = train_df[[
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked',
    'Title'
]]
target = train_df['Survived']

# カテゴリ変数のダミー変数化
# Pclass, Sex, Embarked, Title
features = pd.get_dummies(features, columns=['Pclass', 'Sex', 'Embarked', 'Title'])

# ------------------------
# 3. 学習用/テスト用に分割
# ------------------------
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=42
)

# ------------------------
# 4. スケーリング
# ------------------------
scaler = StandardScaler()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

# ------------------------
# 5. GradientBoostingClassifier + GridSearchCV
# ------------------------
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5],
    'min_samples_split': [2, 5],
}
gb_for_search = GradientBoostingClassifier(random_state=42)

grid_search = GridSearchCV(
    gb_for_search,
    param_grid,
    cv=5,        # 交差検証5分割
    n_jobs=-1,   # 並列実行（PC環境により変更）
    verbose=1
)
grid_search.fit(features_train_scaled, target_train)

print("\n===== GridSearchCV 結果 =====")
print("Best Params:", grid_search.best_params_)
print("CV Best Score:", grid_search.best_score_)

# ベストモデルでテストデータを評価
best_model = grid_search.best_estimator_
test_score = best_model.score(features_test_scaled, target_test)
print(f"Test Score (with best params): {test_score:.4f}")


<ipython-input-147-b82fb851d564>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
<ipython-input-147-b82fb851d564>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

Fitting 5 folds for each of 24 candidates, totalling 120 fits

===== GridSearchCV 結果 =====
Best Params: {'learning_rate': 0.05, 'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100}
CV Best Score: 0.8255462184873948
Test Score (with best params): 0.8271


In [148]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

# xgboost
from xgboost import XGBClassifier

# ------------------------
# 1. データ読み込み
# ------------------------
train_df = pd.read_csv('data/train.csv')  # KaggleのTitanic練習用データ
test_df = pd.read_csv('data/test.csv')    # 提出用（今回精度計測には使わない）
# 実際のKaggle提出をする場合は、test_dfにも同様の前処理を行う必要があります。

# ------------------------
# 2. 特徴量エンジニアリング
# ------------------------
train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
train_df['Embarked'].fillna('S', inplace=True)

def extract_title(name):
    title = name.split(",")[1].split(".")[0].strip()
    return title

train_df['Title'] = train_df['Name'].apply(extract_title)
common_titles = ['Mr', 'Mrs', 'Miss', 'Master']
train_df.loc[~train_df['Title'].isin(common_titles), 'Title'] = 'Others'

train_df['Fare'].fillna(train_df['Fare'].median(), inplace=True)

features = train_df[[
    'Pclass',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Fare',
    'Embarked',
    'Title'
]]
target = train_df['Survived']

features = pd.get_dummies(features, columns=['Pclass', 'Sex', 'Embarked', 'Title'])

# ------------------------
# 3. 学習用/テスト用に分割
# ------------------------
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.33, random_state=42
)

# ------------------------
# 4. スケーリング
# ------------------------
scaler = StandardScaler()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train)
features_test_scaled = scaler.transform(features_test)

# ------------------------
# 5. XGBoost (XGBClassifier) + GridSearchCV
# ------------------------
# XGBClassifierのパラメータ例
#   - n_estimators: 決定木を何本作るか
#   - learning_rate: 学習率
#   - max_depth: 木の深さ
#   - min_child_weight: 木を分割する際の葉あたり最小サンプル重み (過学習対策)
#   - colsample_bytree: 特徴量のサンプリング率 (過学習対策)

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5],
    'min_child_weight': [1, 5],
    'colsample_bytree': [1.0, 0.8],
}

# 注意:
#  - use_label_encoder=False と eval_metric を設定しないと警告が出るケースあり
xgb_model = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

grid_search = GridSearchCV(
    xgb_model,
    param_grid,
    cv=5,       # 交差検証5分割
    n_jobs=-1,  # 並列実行
    verbose=1
)
grid_search.fit(features_train_scaled, target_train)

print("\n===== GridSearchCV 結果 =====")
print("Best Params:", grid_search.best_params_)
print("CV Best Score:", grid_search.best_score_)

# ベストモデルでテストデータを評価
best_model = grid_search.best_estimator_
test_score = best_model.score(features_test_scaled, target_test)
print(f"Test Score (with best params): {test_score:.4f}")


Fitting 5 folds for each of 48 candidates, totalling 240 fits


<ipython-input-148-acfc3c7fd72f>:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['Age'].fillna(train_df['Age'].median(), inplace=True)
<ipython-input-148-acfc3c7fd72f>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac


===== GridSearchCV 結果 =====
Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 200}
CV Best Score: 0.832282913165266
Test Score (with best params): 0.8237


In [165]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.9 MB/s eta 0:00:00


In [167]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# CatBoostを追加
from catboost import CatBoostClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# グローバル設定
RANDOM_STATE = 42

def load_data(train_path='data/train.csv', test_path='data/test.csv'):
    """
    Titanicのトレーニングデータとテストデータを読み込む。
    今回はtest_pathも読み込むが、提出用には同様の前処理が必要。
    """
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    return train_df, test_df

def fill_missing_values(df):
    """
    Titanicのデータフレームに対して、簡易的な欠損値補完を行う。
    """
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna('S', inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    return df

def extract_title(name):
    """
    例: "Braund, Mr. Owen Harris" -> "Mr"
    """
    return name.split(",")[1].split(".")[0].strip()

def feature_engineering(df):
    """
    タイトル抽出、AgeBand生成、FamilySize/IsAlone生成などの特徴量を作成。
    """
    # Title
    df['Title'] = df['Name'].apply(extract_title)
    common_titles = ['Mr', 'Mrs', 'Miss', 'Master']
    df.loc[~df['Title'].isin(common_titles), 'Title'] = 'Others'

    # AgeBand
    def make_age_band(age):
        if age < 16:
            return 0
        elif age < 32:
            return 1
        elif age < 48:
            return 2
        elif age < 64:
            return 3
        else:
            return 4
    df['AgeBand'] = df['Age'].apply(make_age_band)

    # FamilySize, IsAlone
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = 0
    df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1

    return df

def make_features_target(df):
    """
    ダミー変数化などを行い、X(features)とy(target)を返す。
    """
    features = df[[
        'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
        'Embarked', 'Title', 'AgeBand', 'FamilySize', 'IsAlone'
    ]]
    target = df['Survived']

    features['AgeBand'] = features['AgeBand'].astype('category')
    features = pd.get_dummies(
        features,
        columns=['Pclass', 'Sex', 'Embarked', 'Title', 'AgeBand']
    )
    return features, target

def define_models():
    """
    指定の順番でモデルを定義:
      1. LogisticReg
      2. LinearSVC
      3. KNeighbors
      4. GradientBoost
      5. RandomForest
      6. XGBoost
      7. LightGBM
      8. CatBoost
    """
    return {
        "LogisticReg": LogisticRegression(random_state=RANDOM_STATE, max_iter=200),
        "LinearSVC": LinearSVC(random_state=RANDOM_STATE),
        "KNeighbors": KNeighborsClassifier(),
        "GradientBoost": GradientBoostingClassifier(random_state=RANDOM_STATE),
        "RandomForest": RandomForestClassifier(random_state=RANDOM_STATE),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=RANDOM_STATE),
        "LightGBM": LGBMClassifier(random_state=RANDOM_STATE),
        "CatBoost": CatBoostClassifier(verbose=0, random_state=RANDOM_STATE),
    }

def define_param_grids():
    """
    ツリー系モデル + CatBoost用のパラメータ候補をGridSearchCVで探索するための辞書を返す。
    """
    return {
        "GradientBoost": {
            "n_estimators": [100, 200],
            "learning_rate": [0.1, 0.05],
            "max_depth": [3, 5],
        },
        "RandomForest": {
            "n_estimators": [100, 200],
            "max_depth": [None, 5, 10],
        },
        "XGBoost": {
            "n_estimators": [100, 200],
            "learning_rate": [0.1, 0.05],
            "max_depth": [3, 5],
            "colsample_bytree": [1.0, 0.8],
        },
        "LightGBM": {
            "n_estimators": [100, 200],
            "learning_rate": [0.1, 0.05],
            "max_depth": [5, 10, -1],  # -1: 無制限
            "colsample_bytree": [1.0, 0.8],
        },
        "CatBoost": {
            "iterations": [100, 200],
            "learning_rate": [0.1, 0.05],
            "depth": [3, 5],
        }
    }

def train_and_evaluate(models, param_grids, X_train_scaled, y_train, X_test_scaled, y_test):
    """
    常に交差検証(CV)を行い、ツリー系モデル+CatBoostはGridSearchCVでパラメータ探索。
      - それ以外はcross_val_scoreでCV平均精度を算出
      - すべて最終的にテストスコアも測定
    """
    from sklearn.model_selection import cross_val_score, KFold

    results = {}
    cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    for name, model in models.items():
        if name in param_grids:
            param_grid = param_grids[name]
            grid_search = GridSearchCV(
                model,
                param_grid,
                cv=cv,
                scoring='accuracy',
                n_jobs=-1,
                verbose=0
            )
            grid_search.fit(X_train_scaled, y_train)

            best_estimator = grid_search.best_estimator_
            best_cv_score = grid_search.best_score_
            test_pred = best_estimator.predict(X_test_scaled)
            test_acc = accuracy_score(y_test, test_pred)

            results[name] = {
                "Best Params": grid_search.best_params_,
                "CV Mean Acc": f"{best_cv_score:.4f}",
                "Test Acc": f"{test_acc:.4f}"
            }
        else:
            cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='accuracy')
            cv_mean_score = cv_scores.mean()

            model.fit(X_train_scaled, y_train)
            test_pred = model.predict(X_test_scaled)
            test_acc = accuracy_score(y_test, test_pred)

            results[name] = {
                "CV Mean Acc": f"{cv_mean_score:.4f}",
                "Test Acc": f"{test_acc:.4f}"
            }
    return results

def main():
    """
    メイン関数:
      1) データ読み込み
      2) 前処理 & 特徴量作成
      3) train_test_split
      4) スケーリング
      5) モデル定義, param_grid定義
      6) 学習 & 評価
      7) 結果を順番に表示 (ソートや時間計測なし)
    """
    # 1) データ読み込み
    train_df, _ = load_data()

    # 2) 前処理 & 特徴量作成
    train_df = fill_missing_values(train_df)
    train_df = feature_engineering(train_df)
    X, y = make_features_target(train_df)

    # 3) train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=RANDOM_STATE
    )

    # 4) スケーリング
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 5) モデル定義 & パラメータ
    models = define_models()
    param_grids = define_param_grids()

    # 6) 学習 & 評価 (CV必須)
    results = train_and_evaluate(models, param_grids, X_train_scaled, y_train, X_test_scaled, y_test)

    # 7) 結果をそのまま表示（ユーザー指定の順番）
    print("===== Model Results (in user-specified order) =====")
    for model_name, vals in results.items():
        if "Best Params" in vals:
            print(f"{model_name}:")
            print(f"  Best Params : {vals['Best Params']}")
            print(f"  CV Mean Acc : {vals['CV Mean Acc']}")
            print(f"  Test Acc    : {vals['Test Acc']}")
        else:
            print(f"{model_name}:")
            print(f"  CV Mean Acc : {vals['CV Mean Acc']}")
            print(f"  Test Acc    : {vals['Test Acc']}")

if __name__ == "__main__":
    main()


<ipython-input-167-0107a147d562>:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-167-0107a147d562>:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', tr

[LightGBM] [Info] Number of positive: 222, number of negative: 374
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000280 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 238
[LightGBM] [Info] Number of data points in the train set: 596, number of used features: 22
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.372483 -> initscore=-0.521578
[LightGBM] [Info] Start training from score -0.521578
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf